# OPTaaS - Advanced Options

### <span style="color:red">Note:</span> To run this notebook, you need an API Key. You can get one <a href="mailto:charles.brecque@mindfoundry.ai">here</a>.

## Connect to the OPTaaS server using your API Key

In [1]:
from mindfoundry.optaas.client.client import OPTaaSClient

client = OPTaaSClient('https://optaas.mindfoundry.ai', '<Your OPTaaS API key>')

## Store additional data in a Task

This can be a JSON, array, string, number or boolean:

In [2]:
from mindfoundry.optaas.client.parameter import FloatParameter

task = client.create_task(
    title='My Task with User-defined Data', 
    parameters=[
        FloatParameter('x', minimum=0, maximum=5),
        FloatParameter('y', minimum=1, maximum=5),
    ],
    user_defined_data={
        'description': 'Lorem ipsum...',
        'tags': ['abc', 'defg']
    }
)

print(task.user_defined_data)

{'description': 'Lorem ipsum...', 'tags': ['abc', 'defg']}


## Warm start

If you've already tried some configurations, you can record the scores upfront, thus giving the optimizer a "warm start":

In [3]:
def scoring_function(x, y):
    return (x * y) - (x / y)

warm_start_values = [
    {'x': 0, 'y': 1},
    {'x': 1, 'y': 2},
    {'x': 0.5, 'y': 1.5},
]
for values in warm_start_values:
    task.add_user_defined_configuration(values, scoring_function(**values))

task.get_results()

[{ 'configuration': 'b26e7923-ba7d-472b-83cb-4be88415a755',
   'id': 1375,
   'score': 0.0,
   'user_defined_data': None},
 { 'configuration': '0ca0760a-4799-4c0b-83d6-b00ab2f2d317',
   'id': 1376,
   'score': 1.5,
   'user_defined_data': None},
 { 'configuration': 'cb83f24e-8e37-4b52-a4b4-20d7f850a4a2',
   'id': 1377,
   'score': 0.4166666666666667,
   'user_defined_data': None}]

## Generate multiple configurations

This can be useful if you want to use parallel computation, i.e. generate several configurations and hand them out to separate workers to calculate scores.

In [4]:
number_of_workers = 5
configurations = task.generate_configurations(number_of_workers)
display(configurations)

[{ 'id': 'd602e91a-0c85-496f-b265-ec912cffc364',
   'type': 'exploration',
   'values': {'x': 2.9135065401063764, 'y': 2.184211007880833}},
 { 'id': 'b14fe5c2-1689-4551-af53-5ed089cb5426',
   'type': 'exploration',
   'values': {'x': 4.198055185406863, 'y': 3.0362078865940085}},
 { 'id': '0ffb39e4-c0a8-49b5-b7ce-bd1b4fdf23c8',
   'type': 'exploration',
   'values': {'x': 1.0634067189966405, 'y': 2.722215093145843}},
 { 'id': '318963b2-0886-4c24-b09c-f1b660521b51',
   'type': 'default',
   'values': {'x': 2.5, 'y': 3.0}},
 { 'id': '68189324-bdd9-43e9-9292-a10cbc3f9c08',
   'type': 'exploration',
   'values': {'x': 2.302720263363641, 'y': 4.4328395724605745}}]

## Store additional data in a Result

This can be a JSON, array, string, number or boolean:

In [5]:
for i, configuration in enumerate(configurations):
    score = scoring_function(**configuration.values)
    display(f'Worker {i} Score {score:.3f}')
    next_configuration = task.record_result(configuration, score, 
                                            user_defined_data={'worker': i})

'Worker 0 Score 5.030'

'Worker 1 Score 11.364'

'Worker 2 Score 2.504'

'Worker 3 Score 6.667'

'Worker 4 Score 9.688'

## Report an error

If you encountered an error while calculating the score for a configuration, you can report it:

In [6]:
error_configuration = task.generate_configurations(1)[0]
next_configuration = task.record_result(error_configuration, 
                                        error='Unexpected error: code 12345')
error_result = task.get_results()[-1]
print(f'Score: {error_result.score}  {error_result.error}')

Score: None  Unexpected error: code 12345


## Get best result and configuration

In [7]:
task.get_best_result_and_configuration()

({ 'configuration': 'b14fe5c2-1689-4551-af53-5ed089cb5426',
   'id': 1379,
   'score': 11.363504313637042,
   'user_defined_data': {'worker': 1}},
 { 'id': 'b14fe5c2-1689-4551-af53-5ed089cb5426',
   'type': 'exploration',
   'values': {'x': 4.198055185406863, 'y': 3.0362078865940085}})

## Get top 5 results and configurations

In [8]:
for result in task.get_results(limit=5, best_first=True):
    configuration = task.get_configuration(result.configuration)
    display(f'Score: {result.score:.3f} for configuration: {configuration.values}')

"Score: 11.364 for configuration: {'x': 4.198055185406863, 'y': 3.0362078865940085}"

"Score: 9.688 for configuration: {'x': 2.302720263363641, 'y': 4.4328395724605745}"

"Score: 6.667 for configuration: {'x': 2.5, 'y': 3.0}"

"Score: 5.030 for configuration: {'x': 2.9135065401063764, 'y': 2.184211007880833}"

"Score: 2.504 for configuration: {'x': 1.0634067189966405, 'y': 2.722215093145843}"

## Resume a completed task
Completing a task means that no further configurations can be generated and no further results can be recorded for it.

In [9]:
task.complete()
task.generate_configurations()

OPTaaSError: Cannot add configurations to a completed task

However, you can resume a completed task if necessary:

In [10]:
task.resume()
task.generate_configurations()

[{ 'id': 'bb6b2d45-3145-4763-8ad2-0bbb347fe85c',
   'type': 'exploitation',
   'values': {'x': 4.99999998, 'y': 4.99999998}}]